In [1]:
import sys
import pickle
import numpy as np
import pandas as pd
import os
import copy
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
sys.path.append('/home/ziniu.wzn/FSPN/fspn')
from Structure.StatisticalTypes import MetaType
from Data_prepare.join_data_preparation import JoinDataPreparator
from Data_prepare.schemas import gen_stats_light_schema
schema = gen_stats_light_schema("/home/ziniu.wzn/stats")
hdf_path = "/home/ziniu.wzn/stats/gen_single_light"
meta_data_path = hdf_path + '/meta_data.pkl'
prep = JoinDataPreparator(meta_data_path, schema, max_table_data=20000000)
for relationship_obj in schema.relationships:
    print(relationship_obj.identifier)

comments.PostId = posts.Id
comments.UserId = users.Id
badges.UserId = users.Id
tags.ExcerptPostId = posts.Id
postLinks.PostId = posts.Id
postLinks.RelatedPostId = posts.Id
postHistory.PostId = posts.Id
postHistory.UserId = users.Id
votes.PostId = posts.Id
votes.UserId = users.Id
posts.OwnerUserId = users.Id


In [3]:
relation_ensemble = [
    ['postHistory.PostId = posts.Id', 'comments.PostId = posts.Id'],
    ['votes.PostId = posts.Id', 'comments.PostId = posts.Id'],
    ['postLinks.RelatedPostId = posts.Id', 'comments.PostId = posts.Id'],
    ['posts.OwnerUserId = users.Id', 'badges.UserId = users.Id'],
    ['comments.UserId = users.Id',  'badges.UserId = users.Id'],
    ['comments.UserId = users.Id',  'votes.UserId = users.Id'],
    ['postHistory.UserId = users.Id'],
    ['tags.ExcerptPostId = posts.Id'],
    ['postLinks.PostId = posts.Id'],
    ['comments.PostId = posts.Id', 'postLinks.PostId = posts.Id']
]
full_join_sizes = [813992, 994819, 233362, 3752652, 15937883, 11407421, 321405, 92412, 95474, 225711]

In [ ]:
sample_size = 10000000
relation_set = relation_ensemble[9]
df_samples, meta_types, null_values, full_join_est = prep.generate_n_samples(
                sample_size, relationship_list=relation_set, post_sampling_factor=10)
print(full_join_est)
print(len(df_samples), len(df_samples.columns))
print(meta_types)

In [ ]:
df = df_samples
columns = list(df.columns)
assert len(columns) == len(meta_types) == len(null_values)
table_set = set()
for relationship in relation_set:
    relationship_obj = schema.relationship_dictionary.get(relationship)
    table_set.add(relationship_obj.start)
    table_set.add(relationship_obj.end)
extra_drop_columns = []
for table_name in table_set:
    extra_drop = schema.table_dictionary[table_name].drop_id_attributes
    extra_drop = [table_name+"."+col_name for col_name in extra_drop]
    extra_drop_columns.extend(extra_drop)
for col in extra_drop_columns:
    if col in columns:
        print(col)
        idx = columns.index(col)
        columns.pop(idx)
        meta_types.pop(idx)
        null_values.pop(idx)
        df = df.drop(col, axis=1)
print(len(df), len(df.columns), len(meta_types), len(null_values))

In [ ]:
sys.path.append('/home/ziniu.wzn/BayesCard')
from Models.Bayescard_BN import Bayescard_BN, build_meta_info
meta_info = build_meta_info(df.columns, null_values)
bn = Bayescard_BN(relation_set, meta_info, full_join_sizes[9])

In [ ]:
bn.build_from_data(df, algorithm="chow-liu", max_parents=1, ignore_cols=['Id'], sample_size=200000)

In [ ]:
model_path = "/home/ziniu.wzn/stats/BayesCard_ensemble/9_CLT.pkl"
pickle.dump(bn, open(model_path, 'wb'), pickle.HIGHEST_PROTOCOL)